<a href="https://colab.research.google.com/github/betoval/learning-tensorflow/blob/master/estimators-fashion-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

**Tensorflow Estimators**

`tf.estimator` provide a high-level API for TensorFlow. Used to simplify machine learning programming.

Steps in developing an estimator model:

1. Acquire the data and create the data functions
2. Create the feature columns
3. Instantiate the Estimator
4. Evaluate the model's performance

We will use the fashion_mnist dataset to demonstrate the use of Estimators.

In [23]:
#We begin by acquiring the data
#we already did a similar example using keras

fashion = tf.keras.datasets.fashion_mnist #28x28 pixel images
(x_train, y_train), (x_test, y_test) = fashion.load_data()
print(type(x_train))
x_train, x_test = x_train/255.0, x_test/255.0 #convert data into floats in the range 0 to 1
y_train, y_test = np.int32(y_train), np.int32(y_test) #convert data into 32-bit integers

learning_rate = 1e-4

<class 'numpy.ndarray'>


**Create training and testing input functions**

training function = When you want to do the batching, shuffling, etc.

We are not specifying the number of epochs. It will run forever.

batch_size=number of images presented in one step.

Data is shuffled before each step

In [0]:
#Create training input function
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={"x": x_train},
                                                            y=y_train,
                                                            num_epochs=None,
                                                            batch_size=50,
                                                            shuffle=True
                                                            )                                                        

In [0]:
#Create testing input function
test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={"x": x_test},
                                                             y=y_test,
                                                             num_epochs=1,
                                                             shuffle=False
                                                             )

**Create feature columns**

This will pass the data to an estimator.

key = unique string, being the name of the column corresponding to the name of the dictionary we previosly defined (is "x" in this case).

shape = 28x28, which is the shape of the fashion_mnist images.

In [0]:
#Create feature columns
feature_columns = [tf.feature_column.numeric_column("x", shape=[28,28])]




**Instantiate estimator**

This will do the classification. It will construct a deep neural network (DNN) for us.

1 argument = the feature we specified earlier.
2 argument = network size.
`n_classes` = number of y labels for our fashion dataset.
`model_dir` = the directory that we save the model parameters in.


In [31]:
#Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256,32],
    optimizer='Adam',
    n_classes=10,
    dropout=0.1,
    model_dir="./tmp/mnist_modelx",
    loss_reduction=tf.losses.Reduction.SUM
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/mnist_modelx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


**Train the model**

In [32]:
classifier.train(input_fn=train_input_fn,steps=10000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into ./tmp/mnist_modelx/model.ckpt.
INFO:tensorflow:Calling checkpoint l

**Evaluate the performance of the model**

In [33]:
accuracy_score=classifier.evaluate(input_fn=test_input_fn)["accuracy"] #we use test because we want to know how our training model performs with the test images
print("\nTest accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-07T19:13:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_modelx/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.43983s
INFO:tensorflow:Finished evaluation at 2020-04-07-19:13:58
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.8793, averag